<a href="https://colab.research.google.com/github/MatheusOrange211/Sirio_Libanes_ICU_Prediction/blob/main/2_Implementa%C3%A7%C3%A3o_de_Modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este trabalho foi desenvolvido por **Matheus Naranjo Corrêa**.

# TESTANDO MODELOS PARA PREDIÇÃO DE PACIENTES QUE PRECISARÃO DE UTI

Conforme solicitado pela direção do Hospital Sírio Libanês ao departamento de Tecnologia e Dados, apresentaremos abaixo uma proposta de solução para o seguinte problema:

**PROBLEMA**
> A pandemia da SARS-COVID-19 (popularmente conhecido como coronavírus), vem causando grandes estresses nos sistemas de saúdes globais. Países com alta taxa de desenvolvimento vêm sofrendo com a falta de leitos de Unidade de Terapia Intensiva (UTI) na internação de seus pacientes, levando equipes médicas a terem que aplicar métodos de escolha severos, dando prioridade para os mais idosos e graves. Contudo, tais métodos não auxiliam na resolução do problema dado a alta taxa de contaminação existente, consequência das ondas de infecção que vêm sendo causadas em um efeito de *onda* ao redor do mundo. <br>
Tal problema afeta também países emergentes e subdesenvolvidos, que geralmente já possuem sistemas de saúde superlotados, como no caso do Brasil. Infelizmente a superlotação e a falta de leitos já sobrecarregou sistemas de vários estados, como no caso do estado do Amazonas ([link da matéria](https://g1.globo.com/am/amazonas/noticia/2021/01/14/secretario-de-saude-do-am-fala-que-estado-vive-colapso-do-plano-logistico.ghtml)), onde pacientes não estão mais conseguindo acesso a UTI, assim como não possuem equipamentos básicos para a manutenção de vida, como oxigênio. <br>
Com base nesses acontecimentos e até mesmo na prevenção de sobrecarga do sistema de saúde das redes privadas, o Hospital Sírio-Libanês, referência internacional em saúde, busca prevenir e até mesmo predizer, com base em dados clínicos de seus pacientes, conforme forem sendo admitidos no ambiente hospitalar, a necessidade ou não de internação nas UTIs nas próximas horas. A ideia por trás disso é conseguir desenvolver um modelo de aprendizagem de máquina, conhecido com **Machine Learning**, que consiga auxiliar a junta médica a tomar decisões referentes a necessidade ou não de internação na UTI para aquele paciente, usando as boas práticas de programação e respeitando a Lei Geral da Proteção de dados, conforme indica a lei federal Lei nº 13.709/2018.

Conforme analisado no notebook [Visualizando os Dados](https://github.com/MatheusOrange211/Sirio_Libanes_ICU_Prediction/blob/main/Visualizando_os_dados_Sirio_Libanes.ipynb), realizamos uma breve exploração acerca dos dados fornecidos pela equipe de pesquisa do hospital, buscando entender o que fora nos enviado e como deveriamos ir trabalhando com os dados. Ficou decidido, com base em análises (figura 4 - Internação em UTI pelo tempo de Admissão: Precisou de UTI?), que o melhor janela de admissão para se trabalhar era de até duas horas, uma vez que com ela, poderiamos tentar predizer mais cedo se o paciente necessitaria ou não de internação na UTI.

Nessa segunda parte do projeto, optaremos por testar modelos de Machine Learning que ajudem no nosso problema de **classificação**. Inicialmente realizaremos a importação dos dados, sua limpeza (com funções criadas no notebook de visualização de dados e outras), separação dos dados para treino e teste, o treinamento com alguns algoritmos de classificação e por fim sua validação e implementação.

# IMPORTANDO BIBLIOTECAS

Importaremos as bibliotecas básicas usadas no desenvolvimento de análises e tratamento de dados.

In [28]:
#Bibliotecas básicas para análises e visualização de dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#---------------------------------------------------------
#biblioteca para medição de tempo de execuções dos modelos
import time 
#---------------------------------------------------------
#biblioteca de Modelos de Machine Learning e ferramentas de auxilio
#---------------------------------------------------------
from sklearn import model_selection
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

# Dados

Os dados fornecidos pela equipe de pesquisa do Hospital Sírio Libanês estão disponíves neste site: [Kaggle - Sírio-libanês](https://www.kaggle.com/S%C3%ADrio-Libanes/covid19). Uma explicação dos dados já fora feita no notebook anterior, logo, passaremos para outra parte.


In [29]:
dados = pd.read_excel("https://github.com/alura-cursos/covid-19-clinical/blob/main/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx?raw=true")
dados.head()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,...,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,0.898990,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.333333,-0.230769,-0.132075,-0.593220,0.535714,0.578947,0.333333,-0.230769,-0.132075,-0.586207,0.535714,0.578947,0.443299,0.0000,-0.025641,-0.500000,0.714286,0.838384,-0.076923,-0.459459,-0.313433,-0.636364,0.246377,0.578947,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
2,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-6,0
3,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.107143,0.736842,NaN,NaN,NaN,NaN,-0.107143,0.736842,NaN,NaN,NaN,NaN,0.318681,0.898990,NaN,NaN,NaN,NaN,-0.275362,0.736842,NaN,NaN,NaN,NaN,-1.000000,-1.000000,NaN,NaN,NaN,NaN,-1.000000,-1.000000,6-12,0
4,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,-1.0,-0.871658,-0.871658,-0.871658,-0.871658,-1.0,-0.863874,-0.863874,-0.863874,-0.863874,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.414634,-0.414634,-0.414634,-0.414634,-1.0,-0.979069,-0.979069,...,-0.996762,-1.0,-0.243021,-0.338537,-0.213031,-0.317859,0.033779,0.665932,-0.283951,-0.376923,-0.188679,-0.379310,0.035714,0.631579,-0.340206,-0.4875,-0.572650,-0.857143,0.098901,0.797980,-0.076923,0.286486,0.298507,0.272727,0.362319,0.947368,-0.33913,0.325153,0.114504,0.176471,-0.238095,-0.818182,-0.389967,0.407558,

#FUNÇÕES BÁSICAS - PREPARANDO OS DADOS

Abaixo temos funções que realizarão as limpezas iniciais dos dados

Função usada para nos mostrar um breve resumo do nosso DataFrame (Usada no notebook Visualizando dados)

In [30]:
def resume_dataframe(dataset  : pd.DataFrame):
  
  data_nan = dataset.isnull().any().any() #retorno os dados Not a Number das colunas
  dataset_types = list (set(dataset.dtypes.values)) #com o set realizo um "filtro" removendo dados repetidos
  print("################ RESUMO BÁSICO ####################\n")
  #shape nos retorna uma tupla com dois valores, sendo um referente a linhas e o outro a coluna, respectivamente.
  print(f"Quantidade de instâncias: {dataset.shape[0]} (linhas)\nQuantidade de Atributos: {dataset.shape[1]} (colunas)\n")

  if data_nan:
    print(f"Possui dados NaN ? {data_nan}\nQuantidade de NaN totais: {dataset.isnull().sum().values.sum()}\n")
  else:
    print("Não há dados ausentes neste dataset\n")

  print(f"Tipos de  dados que temos :\n{dataset_types}\n")
  print("##################################################\n")

Essa função divide nosso dataframe em 3 pedaços:
* features continuas (contém grande parte das colunas de dados clínicos e NaN)
* features categoricas (as 13 primeiras colunas do nosso DataFrame)
* saida (as duas últimas colunas  - `WINDOW`e `ICU`)

Aplica-se groupby para agrupar cada valor da coluna `PATIENT_VISIT_IDENTIFIER` e nela, aplicar, para as colunas continuas selecionadas, os métodos `bfill`e `fill` para assim preencher os dados faltantesm uma vez que por serem relacionados a saúde, não apresentam, no geral, grandes discrepâncias. Por último, agrupamos tudo novamente e reajustamos as colunas, por fim retornando os dados.

In [31]:
def preenche_tabela(dados):
    features_continuas_colunas = dados.iloc[:, 13:-2].columns
    features_continuas = dados.groupby("PATIENT_VISIT_IDENTIFIER", as_index=False)[features_continuas_colunas]\
                          .fillna(method='bfill')\
                          .fillna(method='ffill')
    features_categoricas = dados.iloc[:, :13]
    saida = dados.iloc[:, -2:]
    dados_finais = pd.concat([features_categoricas, features_continuas, saida], ignore_index=True,axis=1)
    dados_finais.columns = dados.columns
    return dados_finais

Como trabalharemos com dados referentes a uma janela de até duas horas após a admissão, buscaremos filtrar e adicionar o valor 1 para pacientes que em algum momento foram para a UTI. Conforme explicado no notebook anterior (caso não tenha visto):<br>
>Essa função é responsável por realizar um filtro do qual busca-se manter valores referentes a janela de até 2 duas horas. Aplicando-se esta função em um groupby, ocorrerá que os dados serão agrupados. Uma vez agrupados, faremos, por meio desta função, uma verificação onde se qualquer uma das linhas conferidas for igual a 1, ou seja, se em um agrupamento do paciente **x** em todos os períodos, se, por exemplo, na janela `0-4`, tivermos o valor de `ICU` == 1  (ou seja, um valor True), será aplicado nessa linha, da primeira janela de admissão (no caso, 0-2), o valor igual a 1 para a coluna ICU. Visualizando:


| WINDOW  |ICU   |   
|---------|------|
| 0-2        | 0  | 
|     2-4    |  0 | 
|       4-6  |   1| 
|       6-12  |   0| 

>Veja como na janela de 4-6 horas, o paciente já foi para UTI. o que esta função fará é por o valor 1 logo na primeira Janela:

| WINDOW  |ICU   |   
|---------|------|
| 0-2        | 1  | 
|     2-4    |  0 | 
|       4-6  |   0| 
|       6-12  |   0|

>E assim, retornaremos apenas a primeira linha.

Essa função agrupa todos os grupos de dados de um paciente e atribui o valor ICU == 1 na janela de até duas horas para assim pordemos trabalhar apenas com pacientes do quadro de até duas horas. Eis o motivo de termos jogado fora dados de pacientes com `ICU = 1` e `WINDOW = 0-2` logo de cara. Se a pessoa já entrou no hospital precisando de UTI, seus dados não servirão para o modelo.

In [32]:
def prepare_window(rows):
    if(np.any(rows["ICU"])):
        rows.loc[rows["WINDOW"]=="0-2", "ICU"] = 1
    return rows.loc[rows["WINDOW"] == "0-2"]

Nessa função, o que acontece é que passando nosso dataframe (contendo NaN e strings - não se preocupe quanto a isso), filtramos as colunas e ficamos apenas com as que são numéricas. Depois passamos o método de correlação e damos um .`pipe()` (com ele, a gente pode aplicar funções no DataFrame, por exemplo). Aplicamos um lambda e passamos um `np.tril()`. O que ele fará é retornar os valores maiores que -1 numa matrix de diagonal `k-therizada` (Leia a [documentação](https://numpy.org/doc/stable/reference/generated/numpy.tril.html) para um maior aprofundamento). Criamos as colunas e index e empilhamos as colunas que são geradas durante a correlação. Passando outro `pipe()`, retornamos apenas os valores que são maiores que o parâmetro que passamos na declaração da função. Por último damos um `query()` para realizar um filtro em valores que estão na coluna `level_0` e não em `level_1` .

In [33]:
def correlated_columns_harrison(dataset, threshold = 0.95):
  df = dataset[dataset.describe().columns]  #fica-se apenas com as colunas que possuem valores numéricos
  return (
      df.corr().pipe(lambda df1: pd.DataFrame(np.tril(df1,k=-1),  
                                              columns = df.columns,
                                              index = df.columns,
                                              )
      ) 
      .stack()
      .rename("pearson")
      .pipe(
          lambda s: s[s.abs() >threshold].reset_index()
      )
      .query("level_0 not in level_1")
  )

Nessa função, iremos criar uma lista com as colunas com valores correlacionados que serão descartadas. Para as duas colunas que ela possui, iremos dropar valores repetidos e adicionar os que sobram a uma lista que é retornada.

In [34]:
def descartar_colunas_correlacionadas(dataset: pd.DataFrame):
    colunas_com_muita_correlacao = [] #cria-se uma lista vazia 
    for valor in dataset['level_0'].drop_duplicates().values: #dropa-se o nome duplicados de colunsa presentes na coluna level_0
      colunas_com_muita_correlacao.append(valor) #adiciona-se os valores na lista
    for valor in dataset['level_1'].drop_duplicates().values:#dropa-se o nome duplicados de colunsa presentes na coluna level_1
      colunas_com_muita_correlacao.append(valor)  #adiciona-se os valores na lista
    return colunas_com_muita_correlacao

Técnica elaborada por Thiago Gonçalves e Alan Spadinni, conhecida popularmente na literatura alurística de programação como **tecnica Gondinni**, que consiste na criação de correlação de todas as linhas e, com exceção das primeiras 4 e últimas duas colunas do dataframe,  gerar a correlação  e transformação dos valores para absoluto. Depois busca-se selecionar os valores do *triângulo superior* que foram gerados nessa matriz de correlação, todos os valores são transformados para 1s, e depois aplicamos um k = 1 (que gera a divisão dos valores do triângulo superior),, é que aplicamos uma transformação para True nos valores que antes eram 1 , e False para os valores  0s. Dessa forma, quando aplicado o `where`, conseguiremos pegar os valores da matriz superior e assim termos as colunas correlacionas, sem ter a parte do triângulo inferior, que funciona como um espelho. <BR>
Por último, realizando um list comprehenssion, buscaremos todas as colunas maiores que `valor_corte` e adicionamos a `excluir` que será  usada para dropar posteriormente todas as colunas maiores que o valor setado para `valor_corte`.

<img src = "https://static.mundoeducacao.uol.com.br/mundoeducacao/conteudo/matriz-triangular-superior.jpg">

Acima temos um exemplo de uma matriz onde de azul são os valores da diagonal (em azul) que sempre serão iguais a 1, pois são as mesmas colunas refletidas uma na outras , como por exemplo  COLUNA_1 X COLUNA_1 , onde ambas são a mesma coluna, logo é obvio que irão ter correlação perfeita igual a 1. <br>
Em preto, temos o chamado triângulo superior, que é o que desejamos manter, e em vermelho, o chamado triângulo inferior, que será descartado por possuir os mesmos resultados do outro triângulo.

In [35]:
def remove_corr_var(dados,valor_corte = .95):

  matriz_corr = dados.iloc[:,4:-2].corr().abs()
  matrix_upper = matriz_corr.where(np.triu(np.ones(matriz_corr.shape),k=1).astype(np.bool))
  excluir  = [ coluna for coluna in matrix_upper.columns if any(matrix_upper[coluna] > valor_corte)]

  return dados.drop(excluir,axis=1)

Essa função irá executar os modelos:
* DummyClassifier
* LogisticRegression
* DecisionTreeClassifier
* KNeighborsClassifier
* GaussianNB
* SVC
* RandomForestClassifier
* XGBClassifier

Passamos apenas o dataset que deverá ser particionado em treino e teste, depois com um laço de repetição, iremos obter os resultados de cada modelo para determinado conjunto de dados passados. A ideia é apenas ter uma visão inicial de como os dados estão funcionando, se logo de cara, já estão treinando bem, por isso não é passado parâmetros mais específicos para cada modelo, o que pode ocasionar a saída de **WARNINGS**. Contudo, pelos motivos explicados, isso é previsto. Por fim, temos a saída de uma lista com os valores AUC, desvio padrão (STD), e o tempo de execução.

In [36]:
def rodar_varios_modelos(dataset):
  
  x_columns = dataset.columns
  x_columns = x_columns.drop(["PATIENT_VISIT_IDENTIFIER"])
  y = dataset["ICU"]
  x = dataset[x_columns].drop(["ICU"], axis=1)
  x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y)

  results = []
  np.random.seed(415645)
  for model in [DummyClassifier,LogisticRegression,DecisionTreeClassifier,KNeighborsClassifier,GaussianNB,
                SVC,RandomForestClassifier,XGBClassifier]:

                start_time  = time.time()

                model_used = model()

                kfold = StratifiedKFold(n_splits=5,shuffle=True)
                val_score = model_selection.cross_val_score(model_used,x,y,scoring="roc_auc",
                                                            cv = kfold,)
                
                results.append(f"{model.__name__:22} AUC:\
                {val_score.mean():.3f} STD: {val_score.std():.2f}     Tempo execução: {time.time() - start_time} seconds")

  return results

# PREPARANDO O DATAFRAME

Antes de aplicar os modelos e ver qual possui melhor custo benefício, vamos preparar nosso dataframe

Cria-se a variável que armazanerá os dados formatados e previamente, limpos. Aqui, iremos também, realizar uma query por todas as colunas as linhas que possuem `WINDOW=='0-2' and ICU==1`, pois essas, serão eliminadas de imediado, dados que seus dados não nos servem. Realizamos um dropna para remover dados NaN restantes, caso haja.

In [37]:
dados_limpos = preenche_tabela(dados)
a_remover = dados_limpos.query("WINDOW=='0-2' and ICU==1")['PATIENT_VISIT_IDENTIFIER'].values
dados_limpos = dados_limpos.query("PATIENT_VISIT_IDENTIFIER not in @a_remover")
dados_limpos = dados_limpos.dropna()
dados_limpos.head()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,...,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,0.898990,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.333333,-0.230769,-0.132075,-0.593220,0.535714,0.578947,0.333333,-0.230769,-0.132075,-0.586207,0.535714,0.578947,0.443299,0.0000,-0.025641,-0.500000,0.714286,0.838384,-0.076923,-0.459459,-0.313433,-0.636364,0.246377,0.578947,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
2,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,-0.243021,-0.338537,-0.213031,-0.317859,-0.107143,0.736842,-0.283951,-0.376923,-0.188679,-0.379310,-0.107143,0.736842,-0.340206,-0.4875,-0.572650,-0.857143,0.318681,0.898990,-0.076923,0.286486,0.298507,0.272727,-0.275362,0.736842,-0.33913,0.325153,0.114504,0.176471,-1.000000,-1.000000,-0.389967,0.407558,-0.230462,0.096774,-1.000000,-1.000000,4-6,0
3,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,-1.0,-0.871658,-0.871658,-0.871658,-0.871658,-1.0,-0.863874,-0.863874,-0.863874,-0.863874,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.414634,-0.414634,-0.414634,-0.414634,-1.0,-0.979069,-0.979069,...,-0.996762,-1.0,-0.243021,-0.338537,-0.213031,-0.317859,-0.107143,0.736842,-0.283951,-0.376923,-0.188679,-0.379310,-0.107143,0.736842,-0.340206,-0.4875,-0.572650,-0.857143,0.318681,0.898990,-0.076923,0.286486,0.298507,0.27272

Possuímos dados Categóricos em nosso conjunto de dados. Logo, ainda precisamos limpar mais, antes de começarmos a aplicar algum modelo. As colunas `AGE_PERCENTIL`e `WINDOW` possuem informações do tipo **String** que precisam ser categorizadas de forma a não gerar uma discrepância alta de valores e assim influenciar de forma errada nosso modelo. Digo isso pois, a aplicação de um `.astype("category").cat.codes` resultaria em valores iguais a 5,3, 2 e assim em diante, e nossos estão normalizados. Buscando resolver isso, aplico um `pd.get_dummies` para que seja criada uma coluna para cada valor, sendo preenchido de 0s e 1s. Observe também que antes de realizar isso, na linha acima, em `apply`, aplico a função `prepare window`. Ficamos então apenas com o quadro de até duas horas após a admissão.

In [38]:
dados_limpos = dados_limpos.groupby("PATIENT_VISIT_IDENTIFIER").apply(prepare_window)
dados_limpos = pd.get_dummies(dados_limpos) 
dados_limpos.head()

,,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,BILLIRUBIN_MIN,...,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,ICU,AGE_PERCENTIL_10th,AGE_PERCENTIL_20th,AGE_PERCENTIL_30th,AGE_PERCENTIL_40th,AGE_PERCENTIL_50th,AGE_PERCENTIL_60th,AGE_PERCENTIL_70th,AGE_PERCENTIL_80th,AGE_PERCENTIL_90th,AGE_PERCENTIL_Above 90th,WINDOW_0-2
PATIENT_VISIT_IDENTIFIER,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,-0.938950,...,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,0.898990,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1,0,0,0,0,0,1,0,0,0,0,1
2,10,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,-0.938950,...,-0.056604,-0.517241,0.357143,0.947368,-0.525773,-0.5125,-0.111111,-0.714286,0.604396,0.959596,-0.435897,-0.491892,0.000000,-0.575758,0.101449,1.000000,-0.547826,-0.533742,-0.603053,-0.764706,-1.000000,-0.959596,-0.515528,-0.351328,-0.747001,-0.756272,-1.000000,-0.961262,1,1,0,0,0,0,0,0,0,0,0,1
3,15,3,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.263158,-0.263158,-0.263158,-0.263158,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.972789,-0.972789,-0.972789,...,-0.528302,-0.448276,-0.285714,0.684211,0.175258,-0.1125,-0.384615,-0.357143,0.208791,0.878788,-0.299145,-0.556757,-0.626866,-0.515152,-0.420290,0.684211,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0,0,0,0,1,0,0,0,0,0,0,1
4,20,4,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.935113,-0.935113,-0.935113,...,0.160377,-0.586207,0.285714,0.868421,0.443299,0.0000,0.196581,-0.571429,0.538462,0.939394,-0.076923,-0.351351,-0.044776,-0.575758,0.072464,0.894737,-1.000000,-0.877301,-0.923664,-0.882353,-0.952381,-0.979798,-1.000000,-0.883669,-0.956805,-0.870968,-0.953536,-0.980333,0,1,0,0,0,0,0,0,0,0,0,1
5,25,5,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073

Antes de iniciar a aplicação dos modelos de Machine Learning, vejamos o que temos nos dados agora:

In [39]:
resume_dataframe(dados_limpos)

################ RESUMO BÁSICO ####################

Quantidade de instâncias: 352 (linhas)
Quantidade de Atributos: 240 (colunas)

Não há dados ausentes neste dataset

Tipos de  dados que temos :
[dtype('uint8'), dtype('float64'), dtype('int64')]

##################################################



Possuimos então 240 colunas no total. Entretanto talvez não seja necessário o uso de todas essas colunas, dado que muitas colunas podem vir a atrapalhar as predições. Criarei dois dataframes, onde ambos buscarão excluir dados com alta correlação, mas onde ambos realizam tal tarefa de forma diferente. 

### 2 MÉTODOS PARA EXCLUIR DADOS CORRELACIONADOS

**Excluindo-se dados correlacionados com a técnica de Matt Harisson**

In [40]:
#cria-se um dataframe com 3 colunas(duas onde os valores são o nome de colunas e a terceira com os valores de correlação > .95)
dados_correlacionados = correlated_columns_harrison(dados_limpos,.95)

#filtra-se de ambas as colunas que possuem nomes aqueles que não se repetem, gerando uma lista
excluir = descartar_colunas_correlacionadas(dados_correlacionados)

#descarta-se as colunas com alta correlação
dados_limpos_sem_corr_tipo_1 = dados_limpos.drop(excluir,axis=1)

In [41]:
resume_dataframe(dados_limpos_sem_corr_tipo_1)

################ RESUMO BÁSICO ####################

Quantidade de instâncias: 352 (linhas)
Quantidade de Atributos: 64 (colunas)

Não há dados ausentes neste dataset

Tipos de  dados que temos :
[dtype('uint8'), dtype('float64'), dtype('int64')]

##################################################



**Excluindo-se dados correlacionados com a técnica de Thiago Gonçalves e Alan Spadinni: técnica Gondinni**

In [42]:
dados_limpos_sem_corr_tipo_2 = remove_corr_var(dados_limpos)

In [43]:
resume_dataframe(dados_limpos_sem_corr_tipo_2)

################ RESUMO BÁSICO ####################

Quantidade de instâncias: 352 (linhas)
Quantidade de Atributos: 109 (colunas)

Não há dados ausentes neste dataset

Tipos de  dados que temos :
[dtype('uint8'), dtype('float64'), dtype('int64')]

##################################################



Nota-se que em um método, temos 64 colunas, no outro, 109. 


### DADOS A SEREM TREINADOS INICIALMENTE

>Por fim, temos três variáveis referentes aos dados de pacientes:

* `DADOS_LIMPOS` = **POSSUE TODAS AS COLUNAS (240)**
* `DADOS_LIMPOS_SEM_CORR_TIPO_1` = **POSSUE COLUNAS SEM ALTA CORRELAÇÃO COM BASE NA TÉCNICA DE MATT HARISSON (64)**
* `DADOS_LIMPOS_SEM_CORR_TIPO_2` = **POSSUE COLUNAS SEM ALTA CORRELAÇÃO COM BASE NA TÉCNICA GONDINNI (109)**

>Todas as variáveis estão sem dados Not a Number (NaN) e serão usadas em todos os modelos para que possamos ver qual se desempenhará melhor.

# IMPLEMENTAÇÃO DE MODELOS DE APRENDIZAGEM DE MÁQUINA

Nosso projeto consiste na classificação de pacientes que devem ir para UTI ou não, com base em uma série de dados referentes a seus exames e sinais vitais. Para isso, nada melhor do que aplicar técnicas de aprendizagem de máquinas voltadas para classificação. Por conceito temos:

>Os modelos de ML para problemas de classificação binária preveem um resultado binário (uma de duas classes possíveis).<br> ([link para referência](https://docs.aws.amazon.com/pt_br/machine-learning/latest/dg/types-of-ml-models.html))

No nosso caso, utilizarei os seguintes modelos:

* DummyClassifier
* LogisticRegression
* DecisionTreeClassifier
* KNeighborsClassifier
* GaussianNB
* SVC
* RandomForestClassifier
* XGBClassifier

Contudo, apesar de uma breve explicação sobre cada um, não trabalharemos com todos. Iremos apenas inicialmente rodá-los e dependendo do desempenho, ficaremos com 4 ou 5 modelos para trabalhar. Logo, algumas coisas a se levar em conta:

* Sua precisão, recall e acucácia;
* Para que o modelo possa ser usado, ele deve possuir um custo-benefício bom o suficiente para entrar em produção. De nada serve um modelo 100% bom em classificar, mas que leva 2 horas para fazer isso (levando em conta que os dados crescerão com o tempo).



Testando inicialmente os conjunto de dados com os modelos, mas sem especificar os parâmetros, temos:<br>
***
***
**OBSERVAÇÃO**: desconsiderar avisos de warning
***
***

Rodando primeiramente para `dados_limpos`

In [44]:
resultados_dados_limpos = rodar_varios_modelos(dados_limpos)

/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarn

Rodando para `dados_limpos_sem_corr_tipo_1`


In [45]:
resultados_sem_corr_tipo_1 = rodar_varios_modelos(dados_limpos_sem_corr_tipo_1)

/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarn

rodando em `dados_limpos_sem_corr_tipo_2`

In [46]:
resultados_sem_corr_tipo_2 =rodar_varios_modelos(dados_limpos_sem_corr_tipo_2)

/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarn

#### Resultados iniciais para vários modelos e quais serão usados

# EXPLICAR PARÂMETROS AUC STD E TEMPO DE EXECUÇÃO

Abaixo, iremos conferir melhor a questão de desempenho dos modelos que foram treinados. Todos eles servem para classificação, contudo, podem não ser o melhor para o que é proposto no nosso projeto. Tendo isso em mente, algumas métricas iniciais são levadas em conta e que podem fazer a diferença.
<br>
O AUC - Area Under Curve ou área sobre a curva, mede a capacidade do modelo de prever uma pontuação maior de exemplos positivos em comparação com os exemplos negativos. Como ela não depende do corte da pontuação, você poderá ter uma ideia da precisão da previsão do modelo a partir da métrica AUC, sem escolher um limite.
<br>
Já o STD ou o desvio padrão é uma medida que indica a dispersão dos dados dentro de uma amostra com relação à média. Assim, quando se calcula o desvio padrão juntamente com a média de diferentes grupos, obtém-se mais informações para avaliar e diferenciar seus comportamentos.
<br>
Por fim, o tempo de execução servirá para termos uma noção de quanto tempo está sendo levado para divisão de dados, treino, predição e teste em cada modelo. Isso é de fundamental importância e será determinante mais pra frente, quando nossa base de dados se tornar maior e mais complexa. Afinal, do que adianta um modelo 100% preciso, mas que demora inúmeras horas para auxiliar em algo tão emergencial como saber se um paciente necessita ou não de UTI?


Vejamos então como o filtro de dados realizados, auxiliarão no resultado final dos modelos.

In [47]:
resultados_dados_limpos

['DummyClassifier        AUC:                0.502 STD: 0.06     Tempo execução: 0.02911972999572754 seconds',
 'LogisticRegression     AUC:                0.740 STD: 0.03     Tempo execução: 0.36710309982299805 seconds',
 'DecisionTreeClassifier AUC:                0.619 STD: 0.03     Tempo execução: 0.1636958122253418 seconds',
 'KNeighborsClassifier   AUC:                0.655 STD: 0.06     Tempo execução: 0.11810517311096191 seconds',
 'GaussianNB             AUC:                0.744 STD: 0.03     Tempo execução: 0.03795027732849121 seconds',
 'SVC                    AUC:                0.769 STD: 0.08     Tempo execução: 0.2043297290802002 seconds',
 'RandomForestClassifier AUC:                0.788 STD: 0.01     Tempo execução: 1.117182731628418 seconds',
 'XGBClassifier          AUC:                0.779 STD: 0.02     Tempo execução: 1.270688533782959 seconds']

In [48]:
resultados_sem_corr_tipo_1

['DummyClassifier        AUC:                0.502 STD: 0.06     Tempo execução: 0.025545597076416016 seconds',
 'LogisticRegression     AUC:                0.712 STD: 0.09     Tempo execução: 0.19325542449951172 seconds',
 'DecisionTreeClassifier AUC:                0.589 STD: 0.03     Tempo execução: 0.04266762733459473 seconds',
 'KNeighborsClassifier   AUC:                0.670 STD: 0.04     Tempo execução: 0.06586146354675293 seconds',
 'GaussianNB             AUC:                0.690 STD: 0.04     Tempo execução: 0.026978731155395508 seconds',
 'SVC                    AUC:                0.694 STD: 0.06     Tempo execução: 0.08562803268432617 seconds',
 'RandomForestClassifier AUC:                0.682 STD: 0.05     Tempo execução: 0.81573486328125 seconds',
 'XGBClassifier          AUC:                0.731 STD: 0.04     Tempo execução: 0.3091394901275635 seconds']

In [49]:
resultados_sem_corr_tipo_2

['DummyClassifier        AUC:                0.502 STD: 0.06     Tempo execução: 0.03227996826171875 seconds',
 'LogisticRegression     AUC:                0.750 STD: 0.04     Tempo execução: 0.24123048782348633 seconds',
 'DecisionTreeClassifier AUC:                0.611 STD: 0.02     Tempo execução: 0.08976912498474121 seconds',
 'KNeighborsClassifier   AUC:                0.698 STD: 0.06     Tempo execução: 0.06900429725646973 seconds',
 'GaussianNB             AUC:                0.731 STD: 0.03     Tempo execução: 0.028427600860595703 seconds',
 'SVC                    AUC:                0.754 STD: 0.09     Tempo execução: 0.11909842491149902 seconds',
 'RandomForestClassifier AUC:                0.793 STD: 0.03     Tempo execução: 0.9046792984008789 seconds',
 'XGBClassifier          AUC:                0.776 STD: 0.02     Tempo execução: 0.5851531028747559 seconds']

Ao que tudo indica, os modelos performaram melhor em:
* `dados_limpos` (onde permaneceram a maior parte das colunas)
* `dados_limpos_sem_corr_tipo_2` (Segundo o método GONDINNI)

Além disso, no geral, os modelos que obtiveram bons resultados :
* Logistic Regression
* GaussianNB
* SVC
* Random Forest Classifier
* XGBClassifier


Portanto, analisaremos estes modelos mais a fundo, apresentando uma breve explicação a cerca de seu funcionamento, e realizaremos alguns testes referentes a acerto, tempo de execução e processamento.

# 📊📖 EXPLICANDO MODELOS ESCOLHIDOS

Dado os modelos escolhidos, explicarei inicialmente um pouco de cada um, buscando não extender demais o tema para não perdermos o foco. Logo depois, iremos escolher quais resolvem melhor nosso problema, testar com tais modelos e determinar quais se saíram melhores, para no final, decidirmos.

## 📕 Logistic Regression

Antes de explicar a **logistic regression** ou **regressão logistica**, é necessário compreender um pouco sobre regressão Linear. A **regressão Linear** é o processo de traçar uma reta através dos dados que são gerados através de um diagrama de dispersão. A partir daí, buscamos traçar uma equação que gere uma reta que nos mostre a relação existente nos dados.<br>
A equação usada para buscar tal reta é :
\begin{equation}
\hat y = \alpha + \beta x + e
\end{equation}

onde:
* **x** : Variável independente que busca explicar y
* **y**: Variável dependente a ser prevista
* $\alpha$ e $\beta$ : são parâmetros de distribuição
* $e$ : erros de medida

Abaixo, temos um exemplo de regressão linear. Os pontos em vermelho representam nossos dados dispersos ao longo de um plano cartesiano em $R^2$ e em azul, temos uma reta que é gerada a partir de uma função que nós criamos derivada da equação de regressão linear.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/LinearRegression.svg/1200px-LinearRegression.svg.png" width=480>


Assim, supondo que nosso objetivo fosse, a partir de uma base de dados, classificar a qualidade dos estudos de alunos de acordo com as horas de estudo e suas respectivas notas, poderiamos aplicar a regressão linear facilmente e assim buscar a probabilidade do aluno tirar uma nota $y$ baseado nas $x$ horas que ficou estudando.

Contudo, e se buscássemos saber qual a probabilidade de uma pessoa comprar um produto. Nossa variável resposta seria *comprar* e *não-comprar*. Com base em algumas variáveis como idade, salario e afins, como poderiamos classificar as pessoas que comprariam ou não o nosso produto, ou no caso do projeto, quais irão ou não precisar de UTI. Esse é um caso em que a regressão linear não nos ajudaria.

Nesse caso, podemos então, aplicar uma regressão logistica, que apesar do nome, é na verdade um modelo linear de classificação. Neste modelo, as probabilidades que descrevem os possíveis resultados de um único ensaio são modeladas com o auxílio de uma função logística.<br>
Equação da função logística:
\begin{equation}
  f(x)={\frac {L}{1+e^{-k(x-x_{0})}}}\end{equation}

onde:
* $x_0$: valor do ponto médio do sigmoide;
* $L$ : valor máximo da curva;
* $K$: a taxa de crescimento logístico ou a inclinação da curva.

Visualmente a função logística é assim:<br>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/320px-Logistic-curve.svg.png">

A regressão logística é um recurso que nos permite estimar
a probabilidade associada à ocorrência de determinado
evento em face de um conjunto de variáveis explanatórias.
Como característica, Busca estimar a probabilidade da variável
dependente assumir um determinado valor em
função dos conhecidos de outras variáveis e os resultados da análise ficam contidos no
intervalo de zero a um.
<br>
Na regressão logística, a probabilidade de ocorrência de um
evento pode ser estimada diretamente. No caso da variável
dependente
Y assumir apenas dois possíveis estados (1 ou 0)
e
haver um conjunto de
p variáveis independentes
X1 , X2 , ... , Xp
, o
modelo de regressão logística pode ser escrito da seguinte forma:

\begin{equation}
P\big(Y = 1 ) = \frac{1}{1 + e^{-g(x)}}
\end{equation}

onde: 
\begin{equation}
g\big(x\big) = B_0 + B_1X_1 + ... B_pX_p
\end{equation}

Abaixo temos um exemplo mostrando a dirença entre regressão linear e regressão logística:

<img src= "https://estatsite.com.br/wp-content/uploads/2018/08/1-3.jpg" width=690>

Os pontos brancos em y=1 e y=0 podem ser interpretados como comprar e não-comprar, ou no nosso caso, precisa de UTI = 0, não precisa de UTI = 1. Veja como  a regressão logistica irá conseguir pegar um maior conjunto de dados.

## 📕 GaussianNB


Implementa o algoritmo **Gaussian Naive Bayes** para classificação. Os métodos são baseados na aplicação do teorema de Bayes com a suposição "ingênua" de independência condicional entre cada par de características dado o valor da variável classe. O teorema de Bayes afirma a seguinte relação, dada a variável de classe e vetor de características dependentes através de $yx_1x_n$.

\begin{equation}
P(y \mid x_1, \dots, x_n) = \frac{P(y) P(x_1, \dots, x_n \mid y)}
                                 {P(x_1, \dots, x_n)}
\end{equation}

Baseado no “Teorema de Bayes”, o modelo foi criado por um matemático inglês, e também ministro presibiteriano, chamado Thomas Bayes (1701 – 1761) para tentar provar a existência de Deus.
<br>
Ele recebe o nome de “naive” (ingênuo) porque desconsidera a correlação entre as variáveis (features). Ou seja, se determinada fruta é rotulada como “Limão”, caso ela também seja descrita como “Verde” e “Redonda”, o algoritmo não vai levar em consideração a correlação entre esses fatores. Isso porque trata cada um de forma independente.<br>
Entre as possibilidades de aplicações está a classificação de um e-mail como SPAM ou Não-SPAM e a identificação de um assunto com base em seu conteúdo.

Segundo a documentação do **Sklearn**:
> *Apesar de suas suposições aparentemente super simplificadas, os classificadores ingênuos de Bayes têm funcionado muito bem em muitas situações do mundo real, famosamente classificação de documentos e filtragem de spam. Eles requerem uma pequena quantidade de dados de treinamento para estimar os parâmetros necessários. [...] Embora  seja conhecido como um classificador decente, é conhecido por ser um estimador ruim, de modo que as saídas de probabilidade não devem ser levadas muito a sério.*

Imagem de exemplo de seu comportamento:

<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_calibration_thumb.png">

## 📕 SVC - Support Vector Classification ou  Classificação vetorial de suporte.

O **SVC**  é uma implementação diferente do mesmo algoritmo, **SVM** (Support Vector Machines ou máquinas vetoriais de suporte) que são  um conjunto de métodos de aprendizagem supervisionados utilizados para classificação, regressão e detecção de outliers. O SVM tem como vantagem:
 * Ser eficaz em espaços de alta dimensão;
 * Ainda eficaz nos casos em que o número de dimensões é maior do que o número de amostras.

O SVM é definido como:
>*é um algoritmo que busca uma linha de separação entre duas classes distintas analisando os dois pontos, um de cada grupo, mais próximos da outra classe. Isto é, o SVM escolhe a reta — também chamada de hiperplano em maiores dimensões— entre dois grupos que se distancia mais de cada um (no caso abaixo, a reta vermelha).*

<img src="https://miro.medium.com/max/563/1*DW6xRZ9ylA3JMnlfNFuuBg.png">

Os modelos SVM irão tentar encontrar uma separação linear entre as amostras do conjunto de dados.
Um exemplo visual:
<img src="https://www.machinecurve.com/wp-content/uploads/2020/05/dataset.png" >

## 📕 Random Forest Classifier

Random Forest ou Florestas aleatórias, é um algoritmo de aprendizagem supervisionado. Pode ser usado tanto para classificação quanto para regressão. É também o algoritmo mais flexível e fácil de usar. Uma floresta é composta por árvores. Diz-se que quanto mais árvores ela tem, mais robusta é uma floresta. Florestas aleatórias criam árvores de decisão em amostras de dados selecionadas aleatoriamente, obtém previsão de cada árvore e seleciona a melhor solução por meio de votação. Ele também fornece um indicador muito bom da importância do recurso. <br>

Como vantagens temos:

* Florestas aleatórias são consideradas como um método altamente preciso e robusto devido ao número de árvores de decisão que participam do processo.
* Não sofre do problema de excesso de adaptação. A principal razão é que leva a média de todas as previsões, o que cancela os vieses.
* O algoritmo pode ser usado tanto em problemas de classificação quanto de regressão.
* Florestas aleatórias também podem lidar com valores perdidos. Existem duas maneiras de lidar com isso: usar valores medianos para substituir variáveis contínuas e calcular a média ponderada de proximidade dos valores perdidos.
* Você pode obter a importância relativa do recurso, o que ajuda na seleção dos recursos mais contribuintes para o classificador.



Podemos ainda plotar uma arvore para ter uma noção de como ocorre a distribuição:

<img src="https://shiring.github.io/machine_learning/2017/03/16/rf_plot_ggraph_files/figure-markdown_github/unnamed-chunk-4-1.png" width=800>

## 📕 XGBoost Classifier

XGBoost é uma biblioteca de aumento de gradiente distribuído otimizada projetada para ser altamente eficiente, flexível e portátil. Ele implementa algoritmos de aprendizagem de máquina sob a estrutura Gradient Boosting. O XGBoost fornece um reforço paralelo de árvores (também conhecido como GBDT, GBM) que resolvem muitos problemas de ciência de dados de forma rápida e precisa. <br>
O mesmo código é executado nos principais ambientes distribuídos (Hadoop, SGE, MPI) e pode resolver problemas além de bilhões de exemplos.
<Br>
Podemos ainda visualizar como ocorre o comportamento dos nossos dados estão sendo usados:<br>
<img src="https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2016/07/XGBoost-Plot-of-Single-Decision-Tree-Left-To-Right.png" width=590>

Para um maior conhecimento técnico sobre a biblioteca, com as explicações matemáticas aprofundadas feitas pelos próprios autores, consulte o [documento oficial](https://arxiv.org/pdf/1603.02754.pdf).

# TESTANDO OS MODELOS INDIVIDUALMENTE

Chegou o momento de testarmos individualmente cada modelo, aplicando parâmetros específicos e buscando extrair o máximo que der de cada um, com o objetivo de aumentar os acertos, usando as duas base de dados:
* `dados_limpos`
* `dados_limpos_sem_corr_tipo_2` 

## LOGISTIC REGRESSION

## GAUSSIANNB

## SVC

## RANDOM FOREST CLASSIFIER

## XGBOOST CLASSIFIER

#REFERÊNCIAS

[Sklearn - Modelos Lineares](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)

[Regressão Logística - Prof. Adriana Silva](https://www.youtube.com/watch?v=dcsZsA_wipE&ab_channel=EstaTiDados)

[Modelos de Predição | Regressão Logística](https://medium.com/turing-talks/turing-talks-14-modelo-de-predi%C3%A7%C3%A3o-regress%C3%A3o-log%C3%ADstica-7b70a9098e43)
[edisciplinas - usp](https://edisciplinas.usp.br/pluginfile.php/3769787/mod_resource/content/1/09_RegressaoLogistica.pdf)

[Sklearn - 1.9 Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes)

[Data Geeks - Classificação com Naive Bayes](https://www.datageeks.com.br/naive-bayes/)


[Modelos de Predição| SVM](https://medium.com/turing-talks/turing-talks-12-classifica%C3%A7%C3%A3o-por-svm-f4598094a3f1)

[Creating a simple binary SVM classifier with python and Scikit-learn](https://www.machinecurve.com/index.php/2020/05/03/creating-a-simple-binary-svm-classifier-with-python-and-scikit-learn/#choosing-a-kernel-function)


[Entendendo classificadores de florestas aleatórias em Python](https://www.datacamp.com/community/tutorials/random-forests-classifier-python)

[XGBoost: A Scalable Tree Boosting System](https://arxiv.org/pdf/1603.02754.pdf)